In [2]:
%pip install pandas numpy

     --------------------------------------- 11.3/11.3 MB 23.3 MB/s eta 0:00:00
     --------------------------------------- 13.1/13.1 MB 26.1 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd 
import numpy as np 

index = pd.MultiIndex.from_product([
    ['2023', '2024'],
    ['Q1', 'Q2', 'Q3', 'Q4'],
], names=['Anne', 'Trimestre'])

df = pd.DataFrame({
    'ventes': np.random.randint(1000, 5000, size=8),
    'profits': np.random.randint(100, 1000, size=8)
}, index=index)

print("DataFrame avec MultiIndex:")
print(df) 


DataFrame avec MultiIndex:
                ventes  profits
Anne Trimestre                 
2023 Q1           1957      147
     Q2           2818      904
     Q3           3889      675
     Q4           1482      146
2024 Q1           4826      243
     Q2           1264      428
     Q3           1012      722
     Q4           4568      969


In [4]:
# Sélection par tuple
ventes_2023_Q1 = df.loc[('2023', 'Q1')]
print("Ventes 2023 Q1 :")
print(ventes_2023_Q1)

# Sélection par niveau avec xs
ventes_2023 = df.xs('2023', level='Annee')
print("\nToutes les ventes 2023:")
print(ventes_2023)

# Slicing avancé
premier_trimestre = df.loc[(slice(None), 'Q1'), :]
print("\nTous les Q1 :")
print(premier_trimestre)

# Sélection conditionnelle multi-niveaux
ventes_elevees = df[df['ventes'] > 3000]
print("\nVentes > 3000:")
print(ventes_elevees)

Ventes 2023 Q1 :
ventes     1957
profits     147
Name: (2023, Q1), dtype: int32


KeyError: 'Level Annee not found'

In [ ]:
# Agrégations avancées avec dictionnaires
aggregations = {
    'ventes': ['sum', 'mean', 'std'],
    'profits': ['sum', lambda x: x.quantile(0.8)]  # 80ème percentile
}

resultat_agg = df.groupby('Annee').agg(aggregations)
print("Agrégations avancées :")
print(resultat_agg)

In [ ]:
# Transformation - maintient la forme originale
df['ventes_moyenne_groupe'] = df.groupby('Anne')['ventes'].transform('mean')
df['ecart_moyenne'] = df['ventes'] - df['ventes_moyenne_groupe']

print("DataFrame avec transformations :")
print(df[['ventes', 'ventes_moyenne_groupe', 'ecart_moyenne']])

# Agrégation avec conditions
def taux_croissance(serie):
    return (serie.iloc[-1] - serie.iloc[0]) / serie.iloc[0] * 100

croissance_annuelle = df.groupby('Anne')['ventes'].agg(taux_croissance)
print("\nTaux de croissance annuel :")
print(croissance_annuelle)

In [5]:
# Création de séries temporelles
dates = pd.date_range('2023-01-01', periods=1000, freq='D')
ts = pd.Series(np.random.randn(1000).cumsum(), index=dates)

print("Série temporelle originale :")
print(f"Période : {ts.index.min()} to {ts.index.max()}")
print(f"Shape : {ts.shape}")

# Resampling mensuel avec agrégations custom
resampled = ts.resample('M').agg(['mean', 'std', lambda x: x.max() - x.min()])
resampled.columns = ['moyenne', 'ecart_type', 'range']

print("\nSérie resamplée (mensuelle) :")
print(resampled.head())

Série temporelle originale :
Période : 2023-01-01 00:00:00 to 2025-09-26 00:00:00
Shape : (1000,)

Série resamplée (mensuelle) :
              moyenne  ecart_type      range
2023-01-31   0.997822    1.357915   5.122666
2023-02-28  -0.276677    1.046680   5.120331
2023-03-31   1.416065    1.986008   7.315177
2023-04-30   6.895735    1.541134   7.236377
2023-05-31  11.398356    4.266958  14.353266


C:\Users\benoi\AppData\Local\Temp\ipykernel_34824\1079517040.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  resampled = ts.resample('M').agg(['mean', 'std', lambda x: x.max() - x.min()])


In [6]:
# Fenêtres glissantes avec calculs complexes
def calcul_tendance(x):
    """Calcule la pente de la tendance linéaire"""
    if len(x) < 2:
        return np.nan
    # Pente de la régression (index 0 car polyfit retourne [pente, ordonnée])
    return np.polyfit(range(len(x)), x, 1)[0]

def efficience_marche(x):
    """Ratio rendement / volatilité (Sharpe simplifié)"""
    # Calcul des rendements : (valeur_actuelle - valeur_precedente) / valeur_precedente
    returns = np.diff(x) / x[:-1]
    
    if len(returns) > 1:
        return np.mean(returns) / np.std(returns)
    else:
        return np.nan

# Application des rolling windows
# Note : raw=True est important ici pour la performance (passe des numpy arrays au lieu de Séries)
rolling_features = pd.DataFrame({
    'mean': ts.rolling(window=30).mean(),
    'volatility': ts.rolling(window=30).std(),
    'trend': ts.rolling(window=30).apply(calcul_tendance, raw=True),
    'efficiency': ts.rolling(window=30).apply(efficience_marche, raw=True)
})

print("\nFeatures sur fenêtres glissantes (30 jours) :")
print(rolling_features.tail())


Features sur fenêtres glissantes (30 jours) :
                 mean  volatility     trend  efficiency
2025-09-22  57.464182    3.343726  0.369369    0.388239
2025-09-23  57.795940    3.260311  0.358657    0.356372
2025-09-24  58.072171    3.120498  0.337821    0.297683
2025-09-25  58.334840    2.946189  0.311847    0.260943
2025-09-26  58.634127    2.839483  0.298539    0.298306


In [7]:
# Création d'un grand dataset
np.random.seed(42)
grand_df = pd.DataFrame({
    'A': np.random.rand(100000),
    'B': np.random.rand(100000),
    'C': np.random.rand(100000)
})

# MAUVAISE APPROCHE : apply row-wise (ligne par ligne)
def methode_lente(row):
    return row['A'] * row['B'] + row['C']

# BONNE APPROCHE : opérations vectorisées (colonne entière d'un coup)
grand_df['resultat_vectorise'] = grand_df['A'] * grand_df['B'] + grand_df['C']

# APPROCHE INTERMÉDIAIRE : apply avec axis=1
grand_df['resultat_apply'] = grand_df.apply(methode_lente, axis=1)

print("Vérification équivalence :")
# np.allclose est utilisé pour comparer des flottants (gère les micro-différences de précision)
print(f"Résultats identiques : {np.allclose(grand_df['resultat_vectorise'], grand_df['resultat_apply'])}")

# Test de performance
import time

def tester_performance():
    # Vectorisé
    start = time.time()
    resultat_vec = grand_df['A'] * grand_df['B'] + grand_df['C']
    temps_vec = time.time() - start

    # Apply
    start = time.time()
    resultat_app = grand_df.apply(methode_lente, axis=1)
    temps_app = time.time() - start

    print(f"\nPerformance :")
    print(f"Vectorisé   : {temps_vec:.4f} s")
    print(f"Apply       : {temps_app:.4f} s")
    # On évite la division par zéro si c'est trop rapide
    if temps_vec > 0:
        print(f"Accélération: {temps_app / temps_vec:.1f}x")

tester_performance()

Vérification équivalence :
Résultats identiques : True

Performance :
Vectorisé   : 0.0000 s
Apply       : 1.4174 s


In [2]:
%pip install numba

     ---------------------------------------- 2.7/2.7 MB 21.9 MB/s eta 0:00:00
     --------------------------------------- 38.1/38.1 MB 26.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import time
from numba import jit

# 1. D'abord, on recrée le DataFrame (la partie manquante)
np.random.seed(42)
grand_df = pd.DataFrame({
    'A': np.random.rand(100000),
    'B': np.random.rand(100000),
    'C': np.random.rand(100000)
})

print("DataFrame recréé avec succès.")

# 2. Ensuite, la partie Numba
@jit(nopython=True)
def calcul_rapide(a, b, c):
    return a * b + c

# Préparation des données (Numba veut des arrays NumPy, pas des Series Pandas)
valeurs_a = grand_df['A'].values
valeurs_b = grand_df['B'].values
valeurs_c = grand_df['C'].values

# Mesure du temps
start = time.time()
# Premier appel (inclut le temps de compilation)
resultat_numba = calcul_rapide(valeurs_a, valeurs_b, valeurs_c)
temps_numba = time.time() - start

print(f"Avec Numba : {temps_numba:.4f} s")

# Stockage du résultat
grand_df['resultat_numba'] = resultat_numba
print(grand_df.head())

DataFrame recréé avec succès.
Avec Numba : 1.1906 s
          A         B         C  resultat_numba
0  0.374540  0.580779  0.282588        0.500113
1  0.950714  0.526972  0.458677        0.959676
2  0.731994  0.351037  0.099215        0.356172
3  0.598658  0.493213  0.446837        0.742103
4  0.156019  0.365097  0.203081        0.260043


In [ ]:
import pandas as pd
import numpy as np

# --- ÉTAPE 1 : CRÉATION DES DONNÉES (Pour que df existe) ---
index = pd.MultiIndex.from_product([
    ['2023', '2024'],
    ['Q1', 'Q2', 'Q3', 'Q4'],
], names=['Anne', 'Trimestre'])

df = pd.DataFrame({
    'ventes': np.random.randint(1000, 5000, size=8),
    'profits': np.random.randint(100, 1000, size=8)
}, index=index)

# On reset l'index pour transformer 'Anne' et 'Trimestre' en vraies colonnes
df = df.reset_index()

print("DataFrame initialisé avec succès.\n")
# -----------------------------------------------------------

# 1. Utilisation de catégories pour les données textuelles répétitives
df_categories = df.copy()
df_categories['Anne'] = df_categories['Anne'].astype('category')

print(f"Économie mémoire sur la colonne 'Anne' : {df['Anne'].nbytes} -> {df_categories['Anne'].nbytes} bytes")

# 2. Query optimization
# query est très utile pour écrire des filtres comme du SQL
resultat_query = df.query("ventes > 3000 and profits > 500")
print("\nRésultat query optimisé :")
print(resultat_query)

# 3. Optimisation globale de la mémoire (int64 -> int32, etc.)
def optimiser_memoire(df_input):
    """Optimise l'usage mémoire d'un DataFrame"""
    # On travaille sur une copie pour ne pas casser l'original
    df_opt = df_input.copy() 
    for col in df_opt.columns:
        col_type = df_opt[col].dtype
        
        if col_type == 'float64':
            df_opt[col] = df_opt[col].astype('float32')
        elif col_type == 'int64': # Sur Windows, c'est souvent int32 par défaut, mais utile sur Linux/Mac
            df_opt[col] = df_opt[col].astype('int32')
            
    return df_opt

df_final = optimiser_memoire(df)

print(f"\nMémoire totale originale : {df.memory_usage(deep=True).sum()} bytes")
print(f"Mémoire totale optimisée : {df_final.memory_usage(deep=True).sum()} bytes")

NameError: name 'df' is not defined